# View light-Curves

- author Sylvie Dagoret-Campagne
- creation date 2024-09-03
- last update 2024-09-03

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor=None, alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
from astropy.stats import SigmaClip


import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'x-large'
plt.rcParams['axes.titlesize'] = 'x-large'
plt.rcParams['xtick.labelsize']= 'x-large'
plt.rcParams['ytick.labelsize']= 'x-large'

import scipy
from scipy.optimize import curve_fit

In [ ]:
#inputfilename = "sources_objectTable-t3864-multibands-o912-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069.csv"

## The two lists:
path_photom ="data/photom/sources_objectTable"
tract = 3864

#inputfilename ="objects-lightcurves-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240414_w_2024_15_PREOPS-5069-tract3864.csv"
inputfilename = "objects-lightcurves-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240513_w_2024_20_PREOPS-5146-tract3864.csv"
inputfullfilename= os.path.join(path_photom,inputfilename) 

df_allsources = pd.read_csv(inputfullfilename,index_col=0)

In [ ]:
df_allsources

In [ ]:
len(df_allsources.obj_index.unique())

In [ ]:
list_of_files = os.listdir(path_photom) 

In [ ]:
all_objnum = []
tag = "^sources_objectTable-t3864-multibands-o(.*)-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240513_w_2024_20_PREOPS-5146.csv$"
for idx,file in enumerate(list_of_files):
    coll = re.findall(tag,file)
    if len(coll)>0:
        all_objnum.append(int(coll[0]))
        
all_objnum = sorted(all_objnum)  

In [ ]:
all_df = []
for idx,idx_obj in enumerate(all_objnum):
    filename = f"sources_objectTable-t3864-multibands-o{idx_obj}-LATISS_runs_AUXTEL_DRP_IMAGING_20230509_20240513_w_2024_20_PREOPS-5146.csv"
    fullfilename = os.path.join(path_photom,filename)
    df = pd.read_csv(fullfilename,index_col=0)
    all_df.append(df)
    

In [ ]:
df = all_df[0]

In [ ]:
df

In [ ]:
len(df_allsources[df_allsources.obj_index == all_objnum[0]] )

In [ ]:
import matplotlib.dates as mdates
month_locator = mdates.MonthLocator(interval=1)
day_locator = mdates.DayLocator(interval=1)
year_month_formatter = mdates.DateFormatter("%Y-%m") 
colors = ["g","r","orange","grey","k"]

for idx,df in enumerate(all_df):
    obj_num = df.obj_index.unique()[0]
    
    df_filt = {}
    for idx_b,b in enumerate(["g","r","i","z","y"]):
        df_filt[b] = df[df.band == b]
        
        
    fig,axs = plt.subplots(5,1,figsize=(12,8),sharex=True)
    
    axes_flat = axs.flatten()
    for idx_b,b in enumerate(["g","r","i","z","y"]):
        X = pd.to_datetime(df_filt[b]['obsStart'].values)
        Y = df_filt[b]['psfMag'].values
        Yerr = df_filt[b]['psfMagErr'].values
        N = len(Y)
        clippedMeanMagArray = scipy.stats.sigmaclip(Y, low=3.0, high=3.0).clipped
        clippedMeanMag = np.mean(clippedMeanMagArray)
        clippedMeanMagErr = np.std(clippedMeanMagArray)/np.sqrt(len(clippedMeanMagArray))
        clippeErrorMagArray = scipy.stats.sigmaclip(Y-clippedMeanMag,low=3.0, high=3.0).clipped
        clippeErrorMagSigma = (np.sqrt(np.sum(clippeErrorMagArray**2))/np.sqrt(len(clippeErrorMagArray)))*1000.

        
        
        label = f" band {b} : mag = {clippedMeanMag:.1f} mag , rms = {clippeErrorMagSigma:.0f} mmag, N= {N}"
        axes_flat[idx_b].errorbar(X,(Y-clippedMeanMag)*1000,yerr=Yerr*1000,fmt='.',lw=0.5,c=colors[idx_b],ecolor=colors[idx_b]) 
        axes_flat[idx_b].scatter(X,Y-clippedMeanMag,marker='o',s=10,c=colors[idx_b],label=label) 
        axes_flat[idx_b].xaxis.set_major_locator(month_locator) # Locator for major axis only.
        axes_flat[idx_b].xaxis.set_minor_locator(day_locator)
        axes_flat[idx_b].xaxis.set_major_formatter(year_month_formatter)
        axes_flat[idx_b].set_xticks(axes_flat[idx_b].get_xticks(), axes_flat[idx_b].get_xticklabels(), rotation=45, ha='right')
        axes_flat[idx_b].grid()
        if idx_b == 0:
            axes_flat[idx_b].set_title(f"psf Magnitude for object {obj_num}")
        elif idx_b == 4:
            axes_flat[idx_b].set_xlabel("date obs") 
        axes_flat[idx_b].set_ylabel("dev (mmag)") 
        axes_flat[idx_b].legend(loc="upper left")
        axes_flat[idx_b].set_ylim(-50,50)
    plt.tight_layout()
    plt.show()
    
  
    
    

In [ ]:
Y